In [1]:
import wordcloud
import jieba
import re
from imageio import imread
import pymongo
import time
from tqdm import *
import DXYspider_Func as df

In [2]:
client = pymongo.MongoClient('localhost', 27017)
db = client.DXY
collection1 = db.Heart_postId
collection2 = db.Heart_postInfo
collection3 = db.Heart_replyInfo
collection4 = db.proxies
collection5 = db.Heart_userInfo

In [66]:
condition = {'createdTimestamp':{'$gt':1640966400,'$lt': 1672502400}}          #每年帖子Subject存储
Note=open('2022_subject.txt',mode='w',encoding="utf-8")                       
entityIds = collection2.find(condition)
for entityId in entityIds:
    Note.write(str(entityId.get('subject'))+'\n')
Note.close()


In [3]:
Note=open('postIds.txt',mode='w')                       #PostId存储
entityIds =collection3.distinct('postId')
for entityId in entityIds:
    Note.write(str(entityId)+'\n')
Note.close()

In [56]:
count = 0                                               #Interaction
postIds = df.txt2list('postIds.txt')   
for i in tqdm(range(len(postIds))):
    data = collection3.find_one({'postId':postIds[0]})  
    replyList = data.get('replyList')
    for reply in replyList:
        #print(reply.get('userId'))
        Id = reply.get('userId')
        collection5.update_one(
            {'userId':Id},
            {'$inc': {'interaction':1}},
            upsert=True)

    if reply.get('talkCount')>0:
        talks = reply.get('reply')
        for talk in talks:
            collection5.update_one(
                {'userId':talk},
                {'$inc': {'interaction':1}},
                upsert=True)
    count+=1
    del postIds[0]                                  #移除postId

    if  count!= 0 and count%100 == 0:                #进度更新
        f=open('postIds.txt',mode='w')                             
        for postId in postIds:
            f.write(str(postId)+'\n')
        f.close()

100%|██████████| 33132/33132 [2:06:07<00:00,  4.38it/s]   


In [73]:
Note=open('userId.txt',mode='w')                       #PostId存储
entityIds =collection5.distinct('userId')
for entityId in entityIds:
    Note.write(str(entityId)+'\n')
Note.close()

In [75]:
count = 0                                               #Interaction
userIds = df.txt2list('userId.txt')   
for i in tqdm(range(len(userIds))):
    userId = userIds[0]
    condition = {'userId':userId}
    bb = collection3.find({'replyList': {'$elemMatch': {'userId': userId}}})
    lastI =0
    firstI =99999999
    for b in bb:
        temp =b.get('postId')
        if temp<firstI:
            firstI = temp
        if temp>lastI:
            lastI = temp
    collection5.update_one(condition,{'$set':{'firstI':firstI,'lastI':lastI}})

    del userIds[0]                                  #移除postId

    if  count!= 0 and count%100 == 0:                #进度更新
        f=open('userId.txt',mode='w')                             
        for postId in userId:
            f.write(str(postId)+'\n')
        f.close()

100%|██████████| 91344/91344 [2:20:39<00:00, 10.82it/s]  
